In [1]:
import pandas as pd

In [29]:
temporadas = pd.read_csv('./dados/premier_league/couk/temporada_atual.csv')
#atual_fbref = pd.read_csv('./dados/italiano/fbref/rodadas_liga.csv')
#anterior = pd.read_csv('./dados/bundesliga/couk/temporadas_anteriores.csv')
#todas = pd.read_csv('./dados/bundesliga/couk/temporadas_baixadas.csv')